# Eat Safe, Love

## Notebook Set Up

In [2]:
# Import dependencies
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our database
collection_list = db.list_collection_names()
print(collection_list)

['establishments']


In [6]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'hygiene': 20}

# Use count_documents to display the number of documents in the result
establishments.count_documents(query)

# Display the first document in the results using pprint
pprint(establishments.find_one(query))


In [ ]:
# Convert the result to a Pandas DataFrame
cursor = establishments.find(query)
df_hygiene_20 = pd.DataFrame(list(cursor))

# Display the number of rows in the DataFrame
count_rows = len(df_hygiene_20.index)
print(f"There are {count_rows} rows in the DataFrame.")

# Display the first 10 rows of the DataFrame
df_hygiene_20.head(10)



### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and 
# has a RatingValue greater than or equal to 4.
query = {'local_authority': 'London', 'rating': {'$gte': 4}}

# Use count_documents to display the number of documents in the result
establishments.count_documents(query)

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

In [ ]:
# Convert the result to a Pandas DataFrame
cursor = establishments.find(query)
df_london_rating_4 = pd.DataFrame(list(cursor))

# Display the number of rows in the DataFrame
count_rows = len(df_london_rating_4.index)

# Display the first 10 rows of the DataFrame
df_london_rating_4.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Query
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5

degree_search = 0.01
latitude = 0.08384000
longitude = 51.49014200

query = {
    'RatingValue': 5,
    'geocode.longitude': {'$exists': True},
    'geocode.latitude': {'$exists': True}
}

# Sort by hygiene score in ascending order and distance to "Penang Flavours"
sort = [('hygiene_score', 1), ('geocode', {
    '$geoNear': {
        '$geometry': {
            'type': 'Point',
            'coordinates': [longitude, latitude]
        }
    }
})]

# Limit to the top 5 results
limit = 5

# Run the query
results = collection.find(query).sort(sort).limit(limit)

# Print the results
for result in results:
    print(result)

In [ ]:
# Convert result to Pandas DataFrame
df_near_Penang = pd.DataFrame(list(results)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipline = [
    {'$match': {'hygiene': 0}},
    {'$group': {'_id': '$local_authority', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
]

# Run the query
result = establishments.aggregate(pipline)

# Print the number of documents in the result
count = sum(result['count'] for result in results)
print(f"There are {count} establishments with hygiene score of 0.")

# Print the first 10 results
result = establishments.aggregate(pipline)
pprint(result[:10])


In [ ]:
# Convert the result to a Pandas DataFrame


# Display the number of rows in the DataFrame


# Display the first 10 rows of the DataFrame
